Structural Equation Modelling (SEM)
1. https://en.wikipedia.org/wiki/Structural_equation_modeling
2. https://www.datacamp.com/tutorial/structural-equation-modeling
3. https://stats.oarc.ucla.edu/r/seminars/rsem/ (R code) https://www.youtube.com/watch?v=sKVFkVoYfbs
4. https://link.springer.com/protocol/10.1007/978-1-4939-7274-6_28
5.
how to read SEM output (cheat sheet)
Loadings (λ): are items good indicators? (≥ .50 is nice; ≥ .70 is strong)
Std. Estimate (β): standardized effect; compare across predictors
z / p-value: is the path different from zero?
R²: variance explained in each outcome
Fit indices: overall model adequacy (CFI/TLI/RMSEA/SRMR)


how good is “good fit”?
Report several indices; look for an overall pattern:
CFI / TLI: ≥ .95 (≥ .90 sometimes acceptable)
RMSEA: ≤ .06–.08 (with 90% CI)
SRMR: ≤ .08

In [12]:
# ========================= SEM (paths only) with subscale constructs =========================
# Models: (1) Delta_oppose, (2) Delta_support, (3) Delta_all (combined)
# pip install semopy==2.3.9 pandas numpy

import pandas as pd
import numpy as np
from semopy import Model, calc_stats

# ------------------------------------------------------------------------------------
# 0) CONFIG — EDIT THESE TO MATCH YOUR CSV
# ------------------------------------------------------------------------------------
CSV_PATH = "final_final_allsubj_personality_coded.csv"   # <-- set your file path

# Subscale / construct columns already in your CSV:
CONSTRUCTS = [
    "BFI_O","BFI_C","BFI_E","BFI_A","BFI_N",
    "NFC_Total",
    "IUS_Pros","IUS_Inhib",
    "IRI_PT","IRI_FS","IRI_EC","IRI_PD",
    "PANAS_PA","PANAS_NA",
    "SVS_Openness","SVS_Conservation"
]

# Outcomes (must be present)
Y_OPPOSE  = "delta_oppose" #delta_oppose
Y_SUPPORT = "delta_support" #delta_support
Y_COMBINED = "Delta_all"     # will be created below

# Flags
STANDARDIZE_CONSTRUCTS = True
USE_ABS_UPDATES = False      # True => |Delta_oppose|, |Delta_support| individually
COMBINED_MODE = "mean_abs_components"  # "mean_abs_components" or abs_of_mean

# ------------------------------------------------------------------------------------
# 1) LOAD & PREP
# ------------------------------------------------------------------------------------
df = pd.read_csv(CSV_PATH)
# rename columns to fit expected names
df = df.rename(columns={'opposing_update': 'delta_oppose', 'supporting_update': 'delta_support'})

# Ensure outcomes exist
for y in [Y_OPPOSE, Y_SUPPORT]:
    if y not in df.columns:
        raise ValueError(f"Outcome column '{y}' is missing from the CSV.")

# Optionally convert each update to absolute magnitude
#if USE_ABS_UPDATES:
#    df[Y_OPPOSE]  = df[Y_OPPOSE].abs()
#    df[Y_SUPPORT] = df[Y_SUPPORT].abs()
#

# Create combined outcome
if COMBINED_MODE == "mean_abs_components":
    # Recommended: average of magnitudes
    df[Y_COMBINED] = (df[Y_OPPOSE].abs() + df[Y_SUPPORT].abs()) / 2.0
elif COMBINED_MODE == "abs_of_mean":
    # Alternative: absolute of the mean
    df[Y_COMBINED] = ((df[Y_OPPOSE] + df[Y_SUPPORT]) / 2.0).abs()
else:
    raise ValueError("COMBINED_MODE must be 'mean_abs_components' or 'abs_of_mean'.")

# Keep only columns we need; drop rows with missing values
available_constructs = [c for c in CONSTRUCTS if c in df.columns]
if not available_constructs:
    raise ValueError("None of the specified CONSTRUCTS were found in the CSV.")

use_cols = available_constructs + [Y_OPPOSE, Y_SUPPORT, Y_COMBINED]
data = df[use_cols].dropna().copy()

# Standardize constructs (not outcomes) for interpretability of standardized paths
if STANDARDIZE_CONSTRUCTS:
    for c in available_constructs:
        s = data[c].std(ddof=0)
        if pd.notnull(s) and s > 0:
            data[c] = (data[c] - data[c].mean()) / s

print(f"n (after dropna) = {len(data)}")
print("Predictors used:", available_constructs)
print("Combined mode:", COMBINED_MODE, "| USE_ABS_UPDATES:", USE_ABS_UPDATES)

# ------------------------------------------------------------------------------------
# 2) Helper: fit one SEM path model for a single outcome
# ------------------------------------------------------------------------------------
def fit_sem_path(data, constructs, outcome):
    """Fits a path model: outcome ~ all constructs (observed)."""
    rhs = " + ".join(constructs)
    syntax = f"{outcome} ~ {rhs}\n"
    m = Model(syntax)
    m.fit(data)

    stats = calc_stats(m)
    est = m.inspect(std_est=True)

    print(est)
    # Extract and sort structural paths by absolute standardized estimate
    betas = est[(est["op"]=="~") & (est["lval"]==outcome)].copy()
    betas["abs_std"] = betas["Est. Std"].abs()
    betas = betas.sort_values("abs_std", ascending=False)

    # Print summary
    print(f"\n=== SEM Path for {outcome} ===")
    #print({k: stats[k] for k in ["CFI","TLI","RMSEA","SRMR","AIC","BIC"]})
    #no latent variables like SRMR, so we'll just print what we have
    fit_indices = ["CFI","TLI","RMSEA","SRMR","AIC","BIC"]
    available_stats = {k: stats[k] for k in fit_indices if k in stats}
    print(available_stats)
    #r2 = stats["r2"].get(outcome, np.nan)

    r2_dict = stats.get("r2", {})      # returns {} if 'r2' key is missing
    r2 = r2_dict.get(outcome, np.nan)  # returns np.nan if outcome not present

    print(f"R^2({outcome}) = {r2:.3f}")
    print("\nStandardized paths (sorted by |Std. Estimate|):")
    print(betas[["rval","Estimate","Std. Err","z-value","p-value","Est. Std"]])

    return {"model": m, "stats": stats, "betas": betas, "r2": r2, "syntax": syntax}

# ------------------------------------------------------------------------------------
# 3) Fit THREE separate models
# ------------------------------------------------------------------------------------
res_opp = fit_sem_path(data, available_constructs, Y_OPPOSE)
res_sup = fit_sem_path(data, available_constructs, Y_SUPPORT)
res_all = fit_sem_path(data, available_constructs, Y_COMBINED)

# ------------------------------------------------------------------------------------
# 4) Save coefficient tables (optional)
# ------------------------------------------------------------------------------------
res_opp["betas"].to_csv("sem_paths_Delta_oppose.csv", index=False)
res_sup["betas"].to_csv("sem_paths_Delta_support.csv", index=False)
res_all["betas"].to_csv("sem_paths_Delta_all.csv", index=False)
print("\nSaved: sem_paths_Delta_oppose.csv / sem_paths_Delta_support.csv / sem_paths_Delta_all.csv")


n (after dropna) = 87
Predictors used: ['BFI_O', 'BFI_C', 'BFI_E', 'BFI_A', 'BFI_N', 'NFC_Total', 'IRI_PT', 'IRI_FS', 'IRI_EC', 'IRI_PD', 'PANAS_PA', 'PANAS_NA', 'SVS_Openness', 'SVS_Conservation']
Combined mode: mean_abs_components | USE_ABS_UPDATES: False
            lval  op              rval  Estimate  Est. Std  Std. Err  \
0   delta_oppose   ~             BFI_O -0.682928 -0.199855  0.441806   
1   delta_oppose   ~             BFI_C  0.805484  0.235720  0.503161   
2   delta_oppose   ~             BFI_E -0.004094 -0.001198  0.431722   
3   delta_oppose   ~             BFI_A -1.597187 -0.467406  0.498915   
4   delta_oppose   ~             BFI_N  0.333198  0.097508  0.437382   
5   delta_oppose   ~         NFC_Total -1.191943 -0.348814  0.375024   
6   delta_oppose   ~            IRI_PT -1.163648 -0.340534  0.542558   
7   delta_oppose   ~            IRI_FS  0.625977  0.183188  0.638564   
8   delta_oppose   ~            IRI_EC  0.929743  0.272083  0.549497   
9   delta_oppose   ~  